In [3]:
library(nlstools)

Warning message:
: package ‘nlstools’ was built under R version 3.2.3
'nlstools' has been loaded.

IMPORTANT NOTICE: Most nonlinear regression models and data set examples
related to predictive microbiolgy have been moved to the package 'nlsMicrobio'



Различной нелинейной регрессии в R много, теже  нейронные сети или kernel ridge regression (a-ka svm для регрессии), деревья решений к ним можно относить.
Мы будем смотреть на более классический методы (то, что нам рассказывает Мелас примерно)

Модель такая:

$y = f(x, \theta) + \varepsilon, \varepsilon \sim N(0, \sigma^2)$

Если параметр $\theta$ входит нелинейно, то у нас не линейная регрессия. Оценка параметра опять производится по методу наименьших квадратов, но теперь хорошей аналитической формулы нету, как, впрочем, и хороших стат. свойств. 

В R для решения задач нелинейно регрессии есть метод nls. В нем для поиска решения используются различные численные методы. Его недостаток — необходимо задавать начальные данные (а это требует понимание модели, чтобы задать хорошее начальное приближение), а также отсутсвие каких-либо полезных сведений про результат.

Кроме того, если задать начальные данные не удачно, то метод просто не сходится — используется метод Ньютона, который не способен пережить гессиан с нулевыми собственными числами (страно, что авторы не используют более адекватные методы. Тем более, метод ньютона еще и не масштабируется на большие объемы данных)

Мы нашли пакет nlstools, который позволяет побороть проблему начального приближения. Для этого в нем есть набор функций, позволяющих визуально «подобрать» удачные начальные данные

Мы в качестве нелинейной модели возьмем простой сигмоид

$ \sigma(x) = \frac{\alpha}{1 + exp(-\beta x + \gamma)}$


In [4]:
sigmoid <- function(x, alpha = 0.1, beta = 0.5, gamma = 3) { alpha / (1.0  + exp(-(x * beta + gamma))) }

In [9]:
model <- formula( y ~ alpha / (1 + exp(-beta * x + gamma)))

Сюда надо скопировать моделирование

In [5]:
? preview

nlstools {nlstools},R Documentation
formula,formula of a non-linear model
data,a data frame with header matching the variables given in the formula
start,a list of parameter starting values which names match the parameters given in the formula
variable,index of the variable to be plotted against the predicted values; default is the first independent variable as it appears in the orginal dataset
x,an object of class 'nls'
smooth,"a logical value, default is FALSE. If smooth is TRUE, a plot of observed values is plotted as a function of 1000 values continuously taken in the range interval [min(variable),max(variable)]. This option can only be used if the number of controlled variables is 1."
xlab,X-label
ylab,Y-label
pch.obs,type of point of the observed values
pch.fit,type of point of the fitted values (not applicable if smooth=TRUE)


Из статьи:  biologically interpretable parameter often allows the user to guess adequate starting values by assessing (often graphically) a set of plausible candidate model parameter values

Будем считать, что у нас такая же ситуация и подставим что-нибудь близкое к реальным параметрам